In [1]:
import pandas as pd

# Chemicals being evaluated

In [2]:
df = pd.read_csv('dossier-evaluation-status-export.csv', sep='\t', skiprows=16)

df = df.drop(columns=['Unnamed: 13'])

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5890 entries, 0 to 5889
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Substance name          5890 non-null   object
 1   Description             390 non-null    object
 2   EC / List no            5890 non-null   object
 3   CAS no                  5890 non-null   object
 4   Decision type           5889 non-null   object
 5   Scope                   5889 non-null   object
 6   Status                  5880 non-null   object
 7   Decision date           2823 non-null   object
 8   Decision's deadline(s)  2790 non-null   object
 9   Decision                2749 non-null   object
 10  Appeal information      71 non-null     object
 11  Dossier url             5754 non-null   object
 12  Latest update           5890 non-null   object
dtypes: object(13)
memory usage: 598.3+ KB


# Substances used in cosmetics

Searched ECHA registered substances for anything used for cosmetics (PC 39)

In [5]:
cosmetics = pd.read_csv('cosmetics-substances-export.csv', sep='\t', skiprows=39)

cosmetics = cosmetics.drop(columns=['Unnamed: 13'])

In [6]:
cosmetics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2838 entries, 0 to 2837
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Name                        2838 non-null   object
 1   EC / List Number            2838 non-null   object
 2   Cas Number                  2838 non-null   object
 3   ID                          2838 non-null   object
 4   Registration Status         2838 non-null   object
 5   Registration Type           2838 non-null   object
 6   Submission Type             2838 non-null   object
 7   Total tonnage Band          2838 non-null   object
 8   Tonnage Band Min            2838 non-null   object
 9   Tonnage Band Max            2838 non-null   object
 10  Last Updated                2838 non-null   object
 11  Factsheet URL               2767 non-null   object
 12  Substance Information Page  2647 non-null   object
dtypes: object(13)
memory usage: 288.4+ KB


In [8]:
cosmetics.iloc[0, :]

Name                                                           (+)-bornan-2-one
EC / List Number                                                      207-355-2
Cas Number                                                             464-49-3
ID                                                                  100.006.688
Registration Status                                                      Active
Registration Type                                                          Full
Submission Type                                                           Joint
Total tonnage Band                                       ≥ 100 to < 1000 tonnes
Tonnage Band Min                                                            100
Tonnage Band Max                                                           1000
Last Updated                                                         16-03-2020
Factsheet URL                 https://echa.europa.eu/registration-dossier/-/...
Substance Information Page    https://ec

In [30]:
cosmetics.iloc[0, -2]

'https://echa.europa.eu/registration-dossier/-/registered-dossier/24627'

In [31]:
cosmetics.iloc[1, -2]

'https://echa.europa.eu/registration-dossier/-/registered-dossier/11919'

In [102]:
df_cos = cosmetics[['EC / List Number', 'ID']]
df_cos.columns = ['EC / List no', 'ID']
df_cos.head()

,EC / List no,ID
0,207-355-2,100.006.688
1,214-275-1,100.012.978
2,239-387-8,100.035.791
3,218-691-4,100.016.993
4,201-766-0,100.001.606


In [103]:
def make_url(x):
    return 'https://echa.europa.eu/brief-profile/-/briefprofile/' + x

In [104]:
df_cos['brief profile'] = df_cos['ID'].copy().map(make_url)

<ipython-input-104-79cd433c4f33>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cos['brief profile'] = df_cos['ID'].copy().map(make_url)


In [105]:
df_cos.drop(columns=['ID'], inplace=True)

/opt/anaconda3/envs/data_engineer_nd/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [106]:
df_cos.head()

,EC / List no,brief profile
0,207-355-2,https://echa.europa.eu/brief-profile/-/briefpr...
1,214-275-1,https://echa.europa.eu/brief-profile/-/briefpr...
2,239-387-8,https://echa.europa.eu/brief-profile/-/briefpr...
3,218-691-4,https://echa.europa.eu/brief-profile/-/briefpr...
4,201-766-0,https://echa.europa.eu/brief-profile/-/briefpr...


In [87]:
used_in_cosmetics = df['EC / List no'].isin(cosmetics['EC / List Number'].unique())

# Merged data

In [40]:
df = pd.merge(df, df_cos, on='EC / List no')

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1862 entries, 0 to 1861
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Substance name          1862 non-null   object
 1   Description             131 non-null    object
 2   EC / List no            1862 non-null   object
 3   CAS no                  1862 non-null   object
 4   Decision type           1862 non-null   object
 5   Scope                   1862 non-null   object
 6   Status                  1862 non-null   object
 7   Decision date           761 non-null    object
 8   Decision's deadline(s)  758 non-null    object
 9   Decision                727 non-null    object
 10  Appeal information      15 non-null     object
 11  Dossier url             1813 non-null   object
 12  Latest update           1862 non-null   object
 13  ID                      1862 non-null   object
dtypes: object(14)
memory usage: 218.2+ KB


In [47]:
df['Latest update'] = pd.to_datetime(df['Latest update'], dayfirst=True)

In [50]:
df.sort_values(by='Latest update', ascending=False).head()

,Substance name,Description,EC / List no,CAS no,Decision type,Scope,Status,Decision date,Decision's deadline(s),Decision,Appeal information,Dossier url,Latest update,ID
0,Camphene,NaN,201-234-8,79-92-5,TPE,Testing Proposal,Ongoing,NaN,NaN,NaN,NaN,https://www.echa.europa.eu/web/guest/registrat...,2021-03-18,100.001.123
1,Camphene,NaN,201-234-8,79-92-5,CCH,Comprehensive,Ongoing,NaN,NaN,NaN,NaN,https://www.echa.europa.eu/web/guest/registrat...,2021-03-18,100.001.123
4,1-aminopropan-2-ol,NaN,201-162-7,78-96-6,TPE,Testing Proposal,Information requested,16/03/2021,21/06/2022,NaN,NaN,https://www.echa.europa.eu/web/guest/registrat...,2021-03-18,100.001.057
5,1-aminopropan-2-ol,NaN,201-162-7,78-96-6,CCH,Comprehensive,Information requested,16/03/2021,24/06/2024,NaN,NaN,https://www.echa.europa.eu/web/guest/registrat...,2021-03-18,100.001.057
6,"Hydrocarbons, C10-14 (even numbered), n-alkane...",NaN,701-035-0,-,TPE,Testing Proposal,Concluded,NaN,NaN,NaN,NaN,https://www.echa.europa.eu/web/guest/registrat...,2021-03-16,100.266.401


# All substances that have non-cosmetic uses

Searched https://echa.europa.eu/information-on-chemicals/registered-substances?p_p_id=dissregisteredsubstances_WAR_dissregsubsportlet&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_pos=1&p_p_col_count=2&_dissregisteredsubstances_WAR_dissregsubsportlet_javax.portlet.action=dissRegisteredSubstancesAction

for all substances with any use besides cosmetics (PC 39).

Exported as 'non-cosmetics-substances-export.csv'.

In [88]:
non_cosmetics = pd.read_csv('non-cosmetics-substances-export.csv', sep='\t', skiprows=78)

In [89]:
non_cosmetics.drop(columns=['Unnamed: 13'], inplace=True)

In [90]:
filt = df['EC / List no'].isin(non_cosmetics['EC / List Number'].unique())

In [91]:
df[~filt & used_in_cosmetics]

,Substance name,Description,EC / List no,CAS no,Decision type,Scope,Status,Decision date,Decision's deadline(s),Decision,Appeal information,Dossier url,Latest update
19,"Star anise, Illicium verum, ext.",Extractives and their physically modified deri...,283-518-1,84650-59-9,TPE,Testing Proposal,Under assessment,NaN,NaN,NaN,NaN,https://www.echa.europa.eu/web/guest/registrat...,15/03/2021
21,Sodium hydrogen N-(1-oxotetradecyl)-L-glutamate,NaN,253-981-4,38517-37-2,TPE,Testing Proposal,Concluded,20/01/2021,NaN,https://www.echa.europa.eu/documents/10162/a5d...,NaN,https://www.echa.europa.eu/web/guest/registrat...,15/03/2021
77,"Propane-1,3-diol",NaN,207-997-3,504-63-2,CCH,Comprehensive,Under assessment,NaN,NaN,NaN,NaN,https://www.echa.europa.eu/web/guest/registrat...,08/03/2021
131,Sodium salts of [[(phosphonomethyl)imino]bis[e...,NaN,701-216-4,-,TPE,Testing Proposal,Follow-up,14/02/2020,19/02/2021,https://www.echa.europa.eu/documents/10162/4b8...,NaN,https://www.echa.europa.eu/web/guest/registrat...,23/02/2021
132,Sodium salts of [[(phosphonomethyl)imino]bis[e...,NaN,701-215-9,-,TPE,Testing Proposal,Follow-up,14/02/2020,19/02/2021,https://www.echa.europa.eu/documents/10162/d75...,NaN,https://www.echa.europa.eu/web/guest/registrat...,23/02/2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4984,Oxydiethylene dibenzoate,NaN,204-407-6,120-55-8,CCH,Targeted,Concluded,22/09/2016,29/09/2017,https://www.echa.europa.eu/documents/10162/8f7...,NaN,https://www.echa.europa.eu/web/guest/registrat...,04/10/2018
4990,"Orange, sweet, ext.",Extractives and their physically modified deri...,232-433-8,8028-48-6,CCH,Targeted,Concluded,20/05/2015,27/08/2015,https://www.echa.europa.eu/documents/10162/870...,NaN,https://www.echa.europa.eu/web/guest/registrat...,04/10/2018
4992,Oligomerisation products of ethylene oxide wit...,NaN,932-164-2,-,TPE,Testing Proposal,Concluded,05/11/2012,05/05/2014,https://www.echa.europa.eu/documents/10162/3f3...,NaN,https://www.echa.europa.eu/web/guest/registrat...,04/10/2018
4993,Oligomerisation products of ethylene oxide wit...,NaN,932-164-2,-,CCH,Targeted,Concluded,01/12/2014,08/12/2015,https://www.echa.europa.eu/documents/10162/fda...,NaN,https://www.echa.europa.eu/web/guest/registrat...,04/10/2018


In [110]:
df_cosmetics_only = pd.merge(df[~filt & used_in_cosmetics], df_cos, on='EC / List no')

In [111]:
df_cosmetics_only.head()

,Substance name,Description,EC / List no,CAS no,Decision type,Scope,Status,Decision date,Decision's deadline(s),Decision,Appeal information,Dossier url,Latest update,brief profile
0,"Star anise, Illicium verum, ext.",Extractives and their physically modified deri...,283-518-1,84650-59-9,TPE,Testing Proposal,Under assessment,NaN,NaN,NaN,NaN,https://www.echa.europa.eu/web/guest/registrat...,15/03/2021,https://echa.europa.eu/brief-profile/-/briefpr...
1,Sodium hydrogen N-(1-oxotetradecyl)-L-glutamate,NaN,253-981-4,38517-37-2,TPE,Testing Proposal,Concluded,20/01/2021,NaN,https://www.echa.europa.eu/documents/10162/a5d...,NaN,https://www.echa.europa.eu/web/guest/registrat...,15/03/2021,https://echa.europa.eu/brief-profile/-/briefpr...
2,Sodium hydrogen N-(1-oxotetradecyl)-L-glutamate,NaN,253-981-4,38517-37-2,CCH,Targeted,Ongoing,NaN,NaN,NaN,NaN,https://www.echa.europa.eu/web/guest/registrat...,31/08/2020,https://echa.europa.eu/brief-profile/-/briefpr...
3,"Propane-1,3-diol",NaN,207-997-3,504-63-2,CCH,Comprehensive,Under assessment,NaN,NaN,NaN,NaN,https://www.echa.europa.eu/web/guest/registrat...,08/03/2021,https://echa.europa.eu/brief-profile/-/briefpr...
4,Sodium salts of [[(phosphonomethyl)imino]bis[e...,NaN,701-216-4,-,TPE,Testing Proposal,Follow-up,14/02/2020,19/02/2021,https://www.echa.europa.eu/documents/10162/4b8...,NaN,https://www.echa.europa.eu/web/guest/registrat...,23/02/2021,https://echa.europa.eu/brief-profile/-/briefpr...


In [113]:
df_cosmetics_only.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 414 entries, 0 to 413
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Substance name          414 non-null    object
 1   Description             15 non-null     object
 2   EC / List no            414 non-null    object
 3   CAS no                  414 non-null    object
 4   Decision type           414 non-null    object
 5   Scope                   414 non-null    object
 6   Status                  414 non-null    object
 7   Decision date           194 non-null    object
 8   Decision's deadline(s)  192 non-null    object
 9   Decision                189 non-null    object
 10  Appeal information      7 non-null      object
 11  Dossier url             406 non-null    object
 12  Latest update           414 non-null    object
 13  brief profile           414 non-null    object
dtypes: object(14)
memory usage: 48.5+ KB


In [114]:
writer = pd.ExcelWriter('cosmetics_only.xlsx')

df_cosmetics_only.to_excel(writer, 'Sheet1')

writer.save()

# Scraping profiles

In [117]:
import requests

In [118]:
url = df_cosmetics_only.iloc[0, -1]

In [137]:
url

'https://echa.europa.eu/brief-profile/-/briefprofile/100.075.899'

In [120]:
test = requests.get(url)

In [128]:
test.headers['Content-Type']

'text/html;charset=UTF-8'

In [129]:
import re

In [143]:
pat = re.compile('This substance is used in the following products: ([\w ,&()\.]+)\.')
re.search(pat, test.text).group(2)

IndexError: no such group

In [140]:
import time

In [141]:
import random

In [142]:
delay = random.randint(2, 5)

In [163]:
def scrape(url):
    time.sleep(random.randint(2, 5))
    response = requests.get(url)
    time.sleep(random.randint(2, 5))
    if random.random() < 0.05:
        time.sleep(30)
    pat = re.compile('This substance is used in the following products: ([\w ,&()-\.]+)\.')
    products = re.search(pat, response.text)
    if products:
        return products.group(1)
    else:
        return 'scrape fail'

In [151]:
to_scrape = df_cosmetics_only['brief profile']

In [152]:
scraped = to_scrape.map(scrape)

In [153]:
scraped

0      washing & cleaning products, biocides (e.g. di...
1      cosmetics and personal care products, inks and...
2      cosmetics and personal care products, inks and...
3      cosmetics and personal care products, fertilis...
4      fertilisers, water softeners, coating products...
                             ...                        
409    air care products, perfumes and fragrances, po...
410    washing & cleaning products, biocides (e.g. di...
411    washing & cleaning products, biocides (e.g. di...
412                 cosmetics and personal care products
413    perfumes and fragrances, polishes and waxes an...
Name: brief profile, Length: 414, dtype: object

In [155]:
filt = scraped == 'cosmetics and personal care products'

In [185]:
df_final = df_cosmetics_only[filt]

In [160]:
df_scrape_fail = df_cosmetics_only[scraped == 'scrape fail'].copy()

In [162]:
df_scrape_fail.iloc[1, -1]

'https://echa.europa.eu/brief-profile/-/briefprofile/100.000.684'

In [164]:
scraped_fix = df_scrape_fail['brief profile'].map(scrape)

In [165]:
scraped_fix

86     lubricants and greases, adhesives and sealants...
87     lubricants and greases, adhesives and sealants...
88     lubricants and greases, adhesives and sealants...
141    coating products, adhesives and sealants and n...
142    coating products, adhesives and sealants and n...
181    hydraulic fluids, adhesives and sealants, coat...
182    hydraulic fluids, adhesives and sealants, coat...
183    hydraulic fluids, adhesives and sealants, coat...
184    hydraulic fluids, adhesives and sealants, coat...
189    lubricants and greases, anti-freeze products, ...
190    lubricants and greases, anti-freeze products, ...
193    lubricants and greases, adhesives and sealants...
195                                          scrape fail
196                                          scrape fail
197                                          scrape fail
198                                          scrape fail
199                                          scrape fail
200                            

In [166]:
scraped_fix[196]

'scrape fail'

In [168]:
df_scrape_fail2 = df_scrape_fail[scraped_fix == 'scrape fail']

In [171]:
df_scrape_fail2['brief profile'].to_list()

['https://echa.europa.eu/brief-profile/-/briefprofile/100.250.726',
 'https://echa.europa.eu/brief-profile/-/briefprofile/100.250.726',
 'https://echa.europa.eu/brief-profile/-/briefprofile/100.099.868',
 'https://echa.europa.eu/brief-profile/-/briefprofile/100.099.868',
 'https://echa.europa.eu/brief-profile/-/briefprofile/100.099.868',
 'https://echa.europa.eu/brief-profile/-/briefprofile/100.099.868',
 'https://echa.europa.eu/brief-profile/-/briefprofile/100.033.327',
 'https://echa.europa.eu/brief-profile/-/briefprofile/100.033.327',
 'https://echa.europa.eu/brief-profile/-/briefprofile/100.007.802',
 'https://echa.europa.eu/brief-profile/-/briefprofile/100.007.802',
 'https://echa.europa.eu/brief-profile/-/briefprofile/100.007.802',
 'https://echa.europa.eu/brief-profile/-/briefprofile/100.045.877',
 'https://echa.europa.eu/brief-profile/-/briefprofile/100.053.143',
 'https://echa.europa.eu/brief-profile/-/briefprofile/100.102.183']

# Fixing links

In [186]:
df_final = df_final.drop(columns=['CAS no'])

In [187]:
def add_link(x):
    return f'<a href="{x}"">{x}</a>'
df_final['brief profile'] = df_final['brief profile'].map(add_link)

AttributeError: 'Series' object has no attribute 'style'

In [177]:
df_final.head()

,Substance name,Description,EC / List no,Decision type,Scope,Status,Decision date,Decision's deadline(s),Decision,Appeal information,Dossier url,Latest update,brief profile
18,"Reaction mass of Ethanaminium, 2-hydroxy-N,N-d...",NaN,951-974-7,TPE,Testing Proposal,Ongoing,NaN,NaN,NaN,NaN,https://www.echa.europa.eu/web/guest/registrat...,15/02/2021,"<a href=""https://echa.europa.eu/brief-profile/..."
19,Strontium hydroxide,NaN,242-367-1,CCH,Comprehensive,Information requested,08/12/2020,15/12/2022,https://www.echa.europa.eu/documents/10162/c97...,NaN,https://www.echa.europa.eu/web/guest/registrat...,08/02/2021,"<a href=""https://echa.europa.eu/brief-profile/..."
20,Strontium hydroxide,NaN,242-367-1,TPE,Testing Proposal,Concluded,NaN,NaN,NaN,NaN,https://www.echa.europa.eu/web/guest/registrat...,13/10/2018,"<a href=""https://echa.europa.eu/brief-profile/..."
38,"Propane-1,2,3-triyl 3,5,5-trimethylhexanoate",NaN,260-257-1,CCH,Comprehensive,Information requested,24/04/2019,"02/11/2021""26/07/2023",https://www.echa.europa.eu/documents/10162/f06...,"<a href=""https://echa.europa.eu/documents/1016...",https://www.echa.europa.eu/web/guest/registrat...,22/01/2021,"<a href=""https://echa.europa.eu/brief-profile/..."
60,Strontium chloride,NaN,233-971-6,CCH,Comprehensive,Concluded,NaN,NaN,NaN,NaN,https://www.echa.europa.eu/web/guest/registrat...,09/12/2020,"<a href=""https://echa.europa.eu/brief-profile/..."


In [188]:
df_final.drop(columns=['Description'], inplace=True)

In [181]:
df_final['Decision'] = df_final['Decision'].map(add_link)

<ipython-input-181-d351916a7c5c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Decision'] = df_final['Decision'].map(add_link)


In [182]:
df_final.head()

,Substance name,EC / List no,Decision type,Scope,Status,Decision date,Decision's deadline(s),Decision,Appeal information,Dossier url,Latest update,brief profile
18,"Reaction mass of Ethanaminium, 2-hydroxy-N,N-d...",951-974-7,TPE,Testing Proposal,Ongoing,NaN,NaN,"<a href=""nan"""">nan</a>",NaN,https://www.echa.europa.eu/web/guest/registrat...,15/02/2021,"<a href=""https://echa.europa.eu/brief-profile/..."
19,Strontium hydroxide,242-367-1,CCH,Comprehensive,Information requested,08/12/2020,15/12/2022,"<a href=""https://www.echa.europa.eu/documents/...",NaN,https://www.echa.europa.eu/web/guest/registrat...,08/02/2021,"<a href=""https://echa.europa.eu/brief-profile/..."
20,Strontium hydroxide,242-367-1,TPE,Testing Proposal,Concluded,NaN,NaN,"<a href=""nan"""">nan</a>",NaN,https://www.echa.europa.eu/web/guest/registrat...,13/10/2018,"<a href=""https://echa.europa.eu/brief-profile/..."
38,"Propane-1,2,3-triyl 3,5,5-trimethylhexanoate",260-257-1,CCH,Comprehensive,Information requested,24/04/2019,"02/11/2021""26/07/2023","<a href=""https://www.echa.europa.eu/documents/...","<a href=""https://echa.europa.eu/documents/1016...",https://www.echa.europa.eu/web/guest/registrat...,22/01/2021,"<a href=""https://echa.europa.eu/brief-profile/..."
60,Strontium chloride,233-971-6,CCH,Comprehensive,Concluded,NaN,NaN,"<a href=""nan"""">nan</a>",NaN,https://www.echa.europa.eu/web/guest/registrat...,09/12/2020,"<a href=""https://echa.europa.eu/brief-profile/..."


In [183]:
df_final.iloc[0, -3]

'https://www.echa.europa.eu/web/guest/registration-dossier/-/registered-dossier/31631'

In [189]:
writer = pd.ExcelWriter('cosmetics_only_scraped.xlsx')
df_final.to_excel(writer, 'Sheet1')
writer.save()